In [6]:

from collections import defaultdict
import numpy as np
import glob
import sys
from sklearn.linear_model import LogisticRegression
from sklearn import svm


def calculate_percentages(true_positives="positive_control_seqs_for_training_8AA.txt"):
    AAs = "GPAVLIMCFYWHKRQNEDST"
    seqs = []
    with open(true_positives) as ofile:
        first =True
        seqlen=0
        for line in ofile:
            line = line.strip()
            if line:
                seqs.append([_ for _ in line])
                if not first:
                    assert len(line) == seqlen
                    first = False
                seqlen = len(line)
    pos_dic = defaultdict(int)
    # give a pseudocount of 1
    for AA in AAs:
        for pos in range(seqlen):
            pos_dic[pos,AA] += 1
    for colpos in range(seqlen):
        for rowpos in range(len(seqs)):
            pos_dic[colpos,seqs[rowpos][colpos]] += 1
    # make pos_dic into percentages
    for entry in pos_dic:
        pos_dic[entry] = pos_dic[entry]/float(len(seqs))
    return dict(pos_dic)

def chemistry_info(chemistry_file="AA_properties.txt"):
    with open(chemistry_file) as fh:
        AA_chemistry = {}
        for line in fh:
            if line.startswith("#"):
                continue
            else:
                line = line.strip().split('\t')
                AA_chemistry[line[0]] = [float(_) for _ in line[1:]]
    return AA_chemistry

def sequence_info(sequence="SEQUENCE", positional_percentages = {("percentages_dict",) : 0.1}, chemistry_dic = {("chemistry_info",) : [0,1,1,0]}):
    seq_info = []
    for pos,AA in enumerate(sequence):
        seq_info.append(positional_percentages[pos,AA])
        seq_info += chemistry_dic[AA]
    return seq_info


def targets_from_phobius(phobius_file='./posteriors/human_STING_posteriors.txt', test_len=8):
    with open(phobius_file) as ofile:
        sequence = ''
        membrane = []
        cytoplasmic = []
        for line in ofile:
            if line.startswith("#"): continue
            line = line.strip().split('\t')
            if line:
                sequence += line[1]
                membrane.append(float(line[4]))
                cytoplasmic.append(float(line[2]))
    end = 0
    for pos in range(len(sequence)-1,-1,-1):
        if membrane[pos] >= 0.5:
            end = pos
            break
    if end == 0:
        return [],[]
    if end + 20 < len(sequence):
        end += 21
    else:
        end = len(sequence)
    boarders = []
    current = []
    for pos in range(end):
        if current or pos == end-1:
            if cytoplasmic[pos] < 0.5 or pos == end-1:
                current.append(pos)
                boarders.append(tuple(current))
                current = []
        elif not current:
            if cytoplasmic[pos] > 0.5:
                current.append(pos)
    if len(boarders[-1]) == 1:
        boarders = boarders[:-1]
    sequences = []
    for boarder in boarders:
        if boarder[1]-boarder[0] < test_len: continue
        sequences.append(sequence[boarder[0]:boarder[1]+1])
    return sequences, boarders

def split_sequence(sequence = "", split_seq = "", length = 8):
    if len(sequence) < len(split_seq):
        return [],[]
    split_positions = [0]
    for pos in range(len(sequence)-len(split_seq)):
        if sequence[pos:pos+len(split_seq)] == split_seq and pos > split_positions[-1]:
            split_positions += [pos, pos+len(split_seq)]
    if split_positions[-1] == 0:
        return [sequence], [(0,len(sequence)-1)]
    split_positions.append(len(sequence))
    assert len(split_positions) % 2 == 0
    split_sequences = [sequence[split_positions[_]:split_positions[_+1]] for _ in range(0, len(split_positions),2)]
    split_seq_positions = [(split_positions[_],split_positions[_+1]) for _ in range(0, len(split_positions),2)]
    return_sequences = []
    return_positions = []
    for pos,seq in enumerate(split_sequences):
        if len(seq) >= length:
            return_sequences.append(split_sequences[pos])
            return_positions.append(split_seq_positions[pos])
    return return_sequences, return_positions


def create_sequences_from_targets(sequence_list = [] , positions_list = [],length=8, avoid = []):
    if avoid:
        for avoided_seq in avoid:
            new_sequence_list = []
            new_positions_list = []
            for pos,sequence in enumerate(sequence_list):
                start = positions_list[pos][0]
                seqs, positions = split_sequence(sequence = sequence, split_seq = avoided_seq, length = length)
                new_sequence_list += seqs
                new_positions_list += [(positions[_][0]+start,positions[_][1]+start) for _ in range(len(positions))]
        sequence_list, positions_list = new_sequence_list, new_positions_list
    new_sequence_list = []
    new_positions_list = []
    for pos, sequence in enumerate(sequence_list):
        start = positions_list[pos][0]
        for startpos in range(len(sequence)-length+1):
            new_sequence_list.append(sequence[startpos:startpos+length])
            new_positions_list.append(( start + startpos, start + startpos + length))
    return new_sequence_list, new_positions_list
'''
def create_training_data(positive_training_example = 'positive_control_seqs_for_training_8AA_dedup.txt',
                         phobius_files = './posteriors', percentage_dic = calculate_percentages(),
                         chemistry_dic=chemistry_info(), training_output_prefix = "training_examples"):
    # Create positive training examples
    positive_seqs = []
    with open(positive_training_example) as fh:
        for line in fh:
            line = line.strip()
            if line:
                positive_seqs.append(line)
    with open(training_output_prefix+".positives",'w') as wfile:
        wfile.write("#Sequence\t{0}\n".format('\t'.join('pos'+str(_)+' percentage\tpos'+str(_)+' size\tpos'+str(_)+
                                                      ' charge\tpos'+str(_)+' hydrophobicity\tpos'+str(_)+' aromatic'
                                                        for _ in range(len(positive_seqs[0])))))
        for seq in positive_seqs:
            wfile.write(seq +'\t'+'\t'.join(str(_) for _ in
                                  sequence_info(sequence=seq, positional_percentages=percentage_dic,
                                                chemistry_dic=chemistry_dic))+'\n')

    # Create negative training examples
    files = glob.glob(phobius_files+'/*')
    negative_seqs = []
    for file in files:
        sequences, boarders = targets_from_phobius(phobius_file=file)
        sequences, boarders = create_sequences_from_targets(sequence_list=sequences, positions_list=boarders,
                                                            avoid=positive_seqs)
        negative_seqs += sequences
    assert len(negative_seqs[0]) == len(positive_seqs[0])
    with open(training_output_prefix+".negatives",'w') as wfile:
        wfile.write("#Sequence\t{0}\n".format('\t'.join('pos'+str(_)+' percentage\tpos'+str(_)+' size\tpos'+str(_)+
                                                      ' charge\tpos'+str(_)+' hydrophobicity\tpos'+str(_)+' aromatic'
                                                        for _ in range(len(negative_seqs[0])))))
        negative_seqs = set(negative_seqs)
        for seq in negative_seqs:
            wfile.write(seq + '\t' + '\t'.join(str(_) for _ in
                                               sequence_info(sequence=seq, positional_percentages=percentage_dic,
                                                             chemistry_dic=chemistry_dic)) + '\n')
'''
def prepare_training_data(training_output_prefix = "training_examples"):
    XOR_list = []
    data_list = []
    with open(training_output_prefix+".positives") as ofile:
        for line in ofile:
            if line.startswith("#"): continue
            XOR_list.append(True)
            line = line.strip().split('\t')
            data_list.append([float(_) for _ in line[1:]])
    with open(training_output_prefix+".negatives") as ofile:
        for line in ofile:
            if line.startswith("#"): continue
            XOR_list.append(False)
            line = line.strip().split('\t')
            data_list.append([float(_) for _ in line[1:]])
    return np.array(XOR_list), np.array(data_list)

def test_sequence_for_cleavage(percentage_dic=calculate_percentages(),
                                   chemistry_dic=chemistry_info(),
                                   phobius_file='./posteriors/human_STING_posteriors.txt',
                                   sequence_list=[],
                                   return_probs=False,
                                   fit_model=LogisticRegression(C=1).fit([[1], [1]], [3, 1]),
                                   return_bool = False,
                                   probability=False):


    if sequence_list:
        sequences = sequence_list
    else:
        sequences, boarders = create_sequences_from_targets(*targets_from_phobius(phobius_file=phobius_file))
    seq_data = []
    for seq in sequences:
        seq_data.append(sequence_info(sequence=seq, positional_percentages=percentage_dic, chemistry_dic=chemistry_dic))
    test_results = fit_model.predict(seq_data)
    if probability:
        probs = fit_model.predict_proba(seq_data)
    if return_probs:
        probs_to_return = []
        for pos, result in enumerate(test_results):
            probs_to_return.append(probs[pos][1])
        return probs_to_return
    if return_bool:
        return test_results
    for pos, result in enumerate(test_results):
        if sequence_list and probability:
            sys.stdout.write('{0} {1} {2}\n'.format(sequences[pos], probs[pos], test_results[pos]))
        elif sequence_list:
            sys.stdout.write('{0} {1}\n'.format(sequences[pos], test_results[pos]))
        elif result and probability:
            sys.stdout.write('{0} {1} {2}\t{3}\t{4}\n'.format(boarders[pos][0], sequences[pos], boarders[pos][1], probs[pos],
                                                   test_results[pos]))
        elif result:
            sys.stdout.write('{0} {1} {2}\n'.format(boarders[pos][0], sequences[pos], boarders[pos][1]))
    return

# this function will be used to test all human genes on the cluster. It will use the
def test_gene_for_cleavage(percentage_dic=calculate_percentages(),
                           chemistry_dic=chemistry_info(),
                           phobius_files='./posteriors'):
    XOR_list, data_list = prepare_training_data()
    SVM = svm.SVC(kernel='rbf',C=10).fit(data_list,XOR_list)
    LR = LogisticRegression(C=10).fit(data_list,XOR_list)
    for phobius_file in glob.glob(phobius_files+'/*'):
        sequences, boarders = create_sequences_from_targets(*targets_from_phobius(phobius_file=phobius_file))
        if not sequences: continue
        seq_data = []
        for seq in sequences:
            seq_data.append(sequence_info(sequence=seq, positional_percentages=percentage_dic, chemistry_dic=chemistry_dic))
        test_results = SVM.predict(seq_data)
        probs = LR.predict_proba(seq_data)
        to_write = ''
        for pos, result in enumerate(test_results):
            if result:
                to_write += '{0} {1} {2}\t{3}\n'.format(boarders[pos][0]+1, sequences[pos], boarders[pos][1]+1,
                                                        probs[pos])
        if to_write:
            outfile = phobius_file.split('/')[-1].split('.')[0]
            with open('./results/' + outfile + '.result', 'w') as wfile:
                wfile.write(to_write)

'''
if __name__ == "__main__":
    test_gene_for_cleavage()
'''




'\nif __name__ == "__main__":\n    test_gene_for_cleavage()\n'

In [8]:
test_gene_for_cleavage(phobius_files='./longest_isoforms')

/Users/kevin/opt/anaconda3/envs/DGAT2/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
from Bio import Entrez

def fetch_gene_name(ncbi_protein_id):
    Entrez.email = "als9294@med.cornell.edu"  # Replace with your email
    handle = Entrez.efetch(db="protein", id=ncbi_protein_id, rettype="gb", retmode="text")
    record = handle.read()
    handle.close()
    
    # Extract gene name from the record (example parsing)
    lines = record.splitlines()
    gene_name = None
    for line in lines:
        if line.startswith("DEFINITION"):
            gene_name = line.split(" ")[-1].strip()
            gene_name = line.replace("DEFINITIION ", "")
            break
    
    return gene_name if gene_name else "Gene name not found"

# Example usage:
protein_id = "NP_000016"
gene_name = fetch_gene_name(protein_id)
print(f"NCBI Protein ID {protein_id} corresponds to gene name: {gene_name}")


NCBI Protein ID NP_000016 corresponds to gene name: DEFINITION  beta-3 adrenergic receptor [Homo sapiens].


In [49]:
#also get gene_ID:

from Bio import Entrez

def fetch_gene_info(ncbi_protein_id):
    Entrez.email = "als9294@med.cornell.edu"  # Replace with your email
    handle = Entrez.efetch(db="protein", id=ncbi_protein_id, rettype="gb", retmode="text")
    record = handle.read()
    handle.close()

    # Initialize variables
    gene_name = None
    gene_id = None

    # Extract gene name and gene ID from the record
    lines = record.splitlines()
    for line in lines:
        if line.strip().startswith("/gene="):
            # Extracting gene name
            gene_name = line.split("=")[1].strip().strip('"')
        if line.strip().startswith("/db_xref=\"GeneID:"):
            # Extracting gene ID
            gene_id = line.split(":")[1].strip().strip('"')

    if gene_name and gene_id:
        return gene_name, gene_id
    elif gene_name:
        return gene_name, "Gene ID not found"
    else:
        return "Gene name not found", "Gene ID not found"

# Example usage:
protein_id = "NP_000016"
gene_name, gene_id = fetch_gene_info(protein_id)
print(f"NCBI Protein ID {protein_id} corresponds to gene name: {gene_name} and gene ID: {gene_id}")


NCBI Protein ID NP_000016 corresponds to gene name: ADRB3 and gene ID: 155


In [25]:
import glob
protein_designations = []
for file in glob.glob('results/*'):
    protein_designations.append(file.split('/')[1].split('.')[0])


In [26]:

def fetch_gene_names(ncbi_protein_ids):
    Entrez.email = "als9294@weill.cornell.edu"  # Replace with your email
    gene_names = {}

    # Batch size for each request
    batch_size = 100  # You can adjust this based on NCBI guidelines

    # Split the protein IDs into batches
    for i in range(0, len(ncbi_protein_ids), batch_size):
        print(f'on batch {i}')
        batch_ids = ncbi_protein_ids[i:i+batch_size]
        batch_id_str = ",".join(batch_ids)
        handle = Entrez.efetch(db="protein", id=batch_id_str, rettype="gb", retmode="text")
        records = handle.read()
        handle.close()

        # Process each record in the batch
        for record in records.split("\n\n"):
            if record.strip():
                lines = record.splitlines()
                protein_id = lines[0].split()[1].strip()
                gene_name = None
                for line in lines:
                    if line.startswith("DEFINITION"):
                        gene_name = line.replace("DEFINITIION ", "")
                        break
                gene_names[protein_id] = gene_name if gene_name else "Gene name not found"

    return gene_names


gene_names = fetch_gene_names(protein_designations)

# Print a subset of the results
for protein_id, gene_name in gene_names.items()[:10]:
    print(f"{protein_id} -> {gene_name}")


on batch 0
on batch 100
on batch 200
on batch 300
on batch 400
on batch 500
on batch 600
on batch 700
on batch 800
on batch 900
on batch 1000
on batch 1100
on batch 1200
on batch 1300
on batch 1400


TypeError: 'dict_items' object is not subscriptable

In [59]:
#also get gene_ID:

from Bio import Entrez

def fetch_gene_info(ncbi_protein_id):
    Entrez.email = "als9294@med.cornell.edu"  # Replace with your email
    handle = Entrez.efetch(db="protein", id=ncbi_protein_id, rettype="gb", retmode="text")
    record = handle.read()
    handle.close()

    # Initialize variables
    gene_name = None
    gene_id = None

    # Extract gene name and gene ID from the record
    lines = record.splitlines()
    for line in lines:
        if line.strip().startswith("/gene="):
            # Extracting gene name
            gene_name = line.split("=")[1].strip().strip('"')
        if line.strip().startswith("/db_xref=\"GeneID:"):
            # Extracting gene ID
            gene_id = line.split(":")[1].strip().strip('"')

    if gene_name and gene_id:
        return gene_name, gene_id
    elif gene_name:
        return gene_name, "Gene ID not found"
    else:
        return "Gene name not found", "Gene ID not found"

def fetch_gene_names_and_IDs(ncbi_protein_ids):
    Entrez.email = "als9294@weill.cornell.edu"  # Replace with your email
    gene_names = {}
    gene_IDs = {}

    # Batch size for each request
    batch_size = 100  # You can adjust this based on NCBI guidelines

    # Split the protein IDs into batches
    for i in range(0, len(ncbi_protein_ids), batch_size):
        #print(f'on batch {i}')
        batch_ids = ncbi_protein_ids[i:i+batch_size]
        #print(f'batch_ids = {batch_ids}')
        batch_id_str = ",".join(batch_ids)
        #print(batch_id_str)
        handle = Entrez.efetch(db="protein", id=batch_id_str, rettype="gb", retmode="text")
        records = handle.read()
        handle.close()

        # Process each record in the batch
        for record in records.split("\n\n"):
            if record.strip():
                lines = record.splitlines()
                protein_id = lines[0].split()[1].strip()
                gene_name = None
                gene_id = None
                for line in lines:
                    if line.strip().startswith("/gene="):
                        # Extracting gene name
                        gene_name = line.split("=")[1].strip().strip('"')
                    if line.strip().startswith("/db_xref=\"GeneID:"):
                        # Extracting gene ID
                        gene_id = line.split(":")[1].strip().strip('"')
                gene_names[protein_id] = gene_name if gene_name else "Gene name not found"
                gene_IDs[protein_id] = gene_id if gene_id else "Gene ID not found"

    return gene_names, gene_IDs

    for line in lines:
        if line.strip().startswith("/gene="):
            # Extracting gene name
            gene_name = line.split("=")[1].strip().strip('"')
        if line.strip().startswith("/db_xref=\"GeneID:"):
            # Extracting gene ID
            gene_id = line.split(":")[1].strip().strip('"')


# Example usage:
protein_id_list = protein_designations
gene_names, gene_ids = fetch_gene_names_and_IDs(protein_id_list)


In [72]:
[gene_names[protein_designations[n]] for n in range(10)]

['OR2AP1',
 'SCN3A',
 'RMND1',
 'HDAC10',
 'GHSR',
 'CATSPERB',
 'SLITRK3',
 'POMK',
 'SLC10A1',
 'KCNA6']

In [73]:
import ast

#results = []
#for file in glob.glob('results/*'):
#    NP_designation = file.split('/')[1].split('.')[0]
#    protein_name = gene_names[NP_designation].replace('DEFINITION  ','')
#    with open(file) as fh:
#        for line in fh:
#            position_site, posteriors = line.strip().split('\t')
#            posteriors = ast.literal_eval(posteriors.replace(' ',',').replace('[', '[').replace(']', ']'))
#            results.append([NP_designation, protein_name, position_site, posteriors])


results = []
for file in glob.glob('results/*'):
    NP_designation = file.split('/')[1].split('.')[0]
    gene_name = gene_names[NP_designation]
    gene_ID = gene_ids[NP_designation]
    #print(NP_designation, gene_name, gene_ID)
    #raise ValueError()
    with open(file) as fh:
        for line in fh:
            position_site, posteriors = line.strip().split('\t')
            posteriors = ast.literal_eval(posteriors.replace(' ',',').replace('[', '[').replace(']', ']'))
            #print(position_site, posteriors)
            #raise ValueError()
            results.append([NP_designation, gene_name, gene_ID, position_site, posteriors])



In [75]:
for N, item in enumerate(sorted(results, key = lambda x: x[-1][0])):
    print(f'{N}\t{item}')

0	['NP_060193', 'TOR4A', '54863', '93 KRRRSRLV 101', [0.00973088, 0.99026912]]
1	['NP_036416', 'KCNH3', '23416', '166 NRRRSRAV 174', [0.01157474, 0.98842526]]
2	['NP_653234', 'KCNH8', '131096', '163 ARRRSRAV 171', [0.01317068, 0.98682932]]
3	['NP_003476', 'GPR68', '8111', '211 AVRRSHGT 219', [0.01525828, 0.98474172]]
4	['NP_054729', 'ASTN2', '23245', '369 SRRRSKGL 377', [0.01544075, 0.98455925]]
5	['NP_000255', 'PTCH1', '5727', '32 RRRRTGGL 40', [0.01727808, 0.98272192]]
6	['NP_001186508', 'SUN2', '25777', '102 RRRRGTGG 110', [0.01729803, 0.98270197]]
7	['NP_001116151', 'TENM2', '57451', '137 KSRRSSGL 145', [0.017712, 0.982288]]
8	['NP_065831', 'DISP3', '57540', '640 KKRRGSGV 648', [0.01780702, 0.98219298]]
9	['NP_036417', 'KCNH4', '23415', '168 ARRRSRTV 176', [0.01856143, 0.98143857]]
10	['NP_004436', 'EPHB6', '2051', '622 RKRRGTGY 630', [0.02105969, 0.97894031]]
11	['NP_066921', 'CACNA1H', '8912', '1583 RRRRSTFP 1591', [0.02405206, 0.97594794]]
12	['NP_997000', 'SLC25A47', '283600', 

In [98]:
#now figure out if the gene is on the ER and filter out if it isn't:

in_all_ER = set()
in_ER_only = set()
lipid_metab_gene = set()
lipid_metabolism_go_terms = set()
go_terms_set = set()
with open('human_GO_mappings.txt') as fh:
    for line in fh:
        if 'endoplasmic reticulum' in line and 'Component' in line:
            line = line.strip().split('\t')
            if line[2] not in go_terms_set:
                #print(line)
                pass
            go_terms_set.add(line[2])
        if 'regulation of lipid metabolic process' in line:
            line = line.strip().split('\t')
            lipid_metabolism_go_terms.add(line[2])
            #print(line)
print(go_terms_set)

#Now map genes to set:
with open('human_GO_mappings.txt') as fh:
    for line in fh:
        if 'endoplasmic reticulum' in line and 'Component' in line:
            line = line.strip().split('\t')
            in_ER.add(line[1])
            if line[2] == 'GO:0005783':
                in_ER_only.add(line[1])
            if line[2] in lipid_metabolism_go_terms:
                lipid_metab_gene.add(line[1])


{'GO:0071458', 'GO:0098556', 'GO:0034663', 'GO:0071556', 'GO:0071782', 'GO:0030867', 'GO:0031205', 'GO:0005788', 'GO:0032541', 'GO:0005790', 'GO:0005791', 'GO:0033116', 'GO:0030176', 'GO:1990578', 'GO:0005793', 'GO:0042406', 'GO:0005789', 'GO:0097038', 'GO:0044322', 'GO:0005783', 'GO:0031227', 'GO:0070971', 'GO:0042175', 'GO:0048237', 'GO:0098554', 'GO:0005786', 'GO:0030868'}


In [103]:
#Now filter out non_ER proteins:
i = 1
for N, item in enumerate(sorted(results, key = lambda x: x[-1][0])):
    if item[2] not in in_ER_only:
        continue
    print(f'{i}\t{item}')
    i += 1

1	['NP_001116151', 'TENM2', '57451', '137 KSRRSSGL 145', [0.017712, 0.982288]]
2	['NP_065831', 'DISP3', '57540', '640 KKRRGSGV 648', [0.01780702, 0.98219298]]
3	['NP_115652', 'RHBDD1', '84236', '1 MQRRSRGI 9', [0.03870975, 0.96129025]]
4	['NP_004590', 'SREBF2', '6721', '340 KRYRSSIN 348', [0.04388577, 0.95611423]]
5	['NP_597999', 'ACER1', '125981', '52 AQKRSRYI 60', [0.06938768, 0.93061232]]
6	['NP_055761', 'SPAST', '6683', '88 AAKRSSGA 96', [0.10227024, 0.89772976]]
7	['NP_004519', 'MGST3', '4259', '109 PSKRSRGA 117', [0.10795652, 0.89204348]]
8	['NP_067060', 'SELENOK', '58515', '47 KKRRSYGN 55', [0.10967241, 0.89032759]]
9	['NP_002819', 'PTPN2', '5771', '252 RKYRMGLI 260', [0.11765713, 0.88234287]]
10	['NP_077816', 'ATP10A', '57194', '50 RRRRGCAQ 58', [0.13019042, 0.86980958]]
11	['NP_112192', 'UNC93B1', '81622', '403 KLRRGVAP 411', [0.1313205, 0.8686795]]
12	['NP_115953', 'DGAT2', '84649', '120 GGRRSQWV 128', [0.13416785, 0.86583215]]
13	['NP_001041695', 'ADORA1', '134', '214 KVSASS

In [101]:
lipid_metabolism_go_terms

{'GO:0019216', 'GO:0045833', 'GO:0045834'}